# neuralmt: default program

In [1]:
from default import *
import os, sys

## Run the default solution on dev

In [2]:
model = Seq2Seq(build=False)
model.load(os.path.join('data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter) # Warning: will take >5mins depending on your machine
print("\n".join(results))

1305it [06:11,  3.51it/s]

i had my own , , i had my own . . 
<unk> was <unk> <unk> , and <unk> <unk> . 
<unk> was a <unk> <unk> . 
and she was , and she was <unk> , and she was <unk> , and she was <unk> , and she <unk> , and she was <unk> , and she <unk> , and she <unk> , and she <unk> . 
i was , i was , i was , i was . 
and so , a first - <unk> , , a <unk> , a <unk> . 
and i was going to talk about the , and i wanted to talk about the <unk> . 
i i , i i , i . 
i did n't have it . 
and , the the , , , the , , , was to the <unk> , , and the <unk> , and the <unk> , and the <unk> , and the <unk> , and it was <unk> to 
i was , , " , " , " , " , " and , " 
later came later , later came later later came later later came later later came later later later came later later later later later later later later later later later later later later later later later later later later later later later later later later later later . 
and i did n't like , like , and i did n't have a <unk> like 
i was to <unk> my <unk> , <unk

## Evaluate the default output

In [4]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results))

BLEU = 2.49 28.3/5.4/1.3/0.4 (BP = 0.854 ratio = 0.864 hyp_len = 21503 ref_len = 24902)


dev score:1.8637

## Change 1

Computation of Energy Scores: The calcAlpha function was updated to compute energy scores by first expanding the decoder hidden state across the sequence length of the encoder outputs to align their dimensions. We then apply the encoder and decoder linear transformations (W_enc and W_dec) to their respective inputs and sum these transformed outputs. The tanh activation function is applied to this sum, followed by another linear transformation (V_att) to produce energy scores. These scores represent the alignment between the encoder outputs and the current decoder hidden state, indicating how much attention to pay to each encoder output at the current decoding step.

Softmax to Calculate Attention Weights: We apply a softmax function to the energy scores along the sequence dimension. This operation converts the scores into a probability distribution (attention weights, alpha), indicating the relative importance of each encoder output in the context of the current decoder state.

Context Vector Calculation: In the forward method, using the calculated attention weights (alpha), we compute the context vector as a weighted sum of the encoder outputs. This context vector synthesizes the information from the encoder outputs, weighted by their relevance to the current step in the decoding process.

In [ ]:
class AttentionModule(nn.Module):
    def __init__(self, attention_dim):
        """
        You shouldn't deleted/change any of the following defs, they are
        essential for successfully loading the saved model.
        """
        super(AttentionModule, self).__init__()
        self.W_enc = nn.Linear(attention_dim, attention_dim, bias=False)
        self.W_dec = nn.Linear(attention_dim, attention_dim, bias=False)
        self.V_att = nn.Linear(attention_dim, 1, bias=False)

    # Start working from here, both 'calcAlpha' and 'forward' need to be fixed
    def calcAlpha(self, decoder_hidden, encoder_out):
        # Expand decoder hidden state to match the sequence length of encoder outputs
        decoder_hidden_expanded = decoder_hidden.permute(1, 0, 2).expand(-1, encoder_out.size(0), -1)
        encoder_out_transposed = encoder_out.permute(1, 0, 2)
        
        # Calculate energy scores
        energy = torch.tanh(self.W_enc(encoder_out_transposed) + self.W_dec(decoder_hidden_expanded))
        scores = self.V_att(energy).squeeze(2)
        
        # Apply softmax to get alpha (attention weights)
        alpha = torch.nn.functional.softmax(scores, dim=1).unsqueeze(1)
        return alpha

    def forward(self, decoder_hidden, encoder_out):
        alpha = self.calcAlpha(decoder_hidden, encoder_out)
        
        # Compute context vector as weighted sum of encoder outputs
        context = torch.bmm(alpha, encoder_out.permute(1, 0, 2)).permute(1, 0, 2)
        return context, alpha

In [4]:
model = Seq2Seq(build=False)
model.load(os.path.join('/home/mihir/nlpclass-1241-g-CtrlAltDefeat/hw3/data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('/home/mihir/nlpclass-1241-g-CtrlAltDefeat/hw3/data', 'input', 'dev.txt'), model.params['srcLex'],device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter) # Warning: will take >5mins depending on your machine
print("\n".join(results))

1305it [32:02,  1.47s/it]

i was in my first time . 
i was in and i . 
she was a woman . 
so she was to , and she was , and she was , and she was , and she was , and she was in the . 
i was i . 
a 's a first . 
i got to talk about the woman . 
i i i . 
i did n't . 
so , the the , the the , , the the , , the the , , the the , , 
i was , , " i , " i said , " 
later , later later later later later later later later later later later . 
i was in the . 
i asked to to , i was . 
i . 
she 's , , " she , , " she 's , , " she . 
but , you know , but you . 
the , the is is that it 's is , " 
what 's what is . 
that was the moment that was that the moment . 
so , the , the the , , 
we were and and and and . 
and the , of the , the , , the , , the , and the was the , and the . 
in the , in the , in the united states . 
so , if you 're going to , you , the percent of the . 
you you you 're . 
i want you to see . 
you 're . 
in fact , i 'll , you , in the , , you know , in the , . 
so , the , of the , , the the , of the , , t

In [8]:
sys.path.append(os.path.dirname(os.path.abspath('/home/mihir/nlpclass-1241-g-CtrlAltDefeat/hw3/answer')))
from bleu_check import bleu

ref_t = []
with open(os.path.join('/home/mihir/nlpclass-1241-g-CtrlAltDefeat/hw3/data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results))

BLEU = 1.86 48.5/9.4/2.3/0.5 (BP = 0.383 ratio = 0.510 hyp_len = 12699 ref_len = 24902)


dev score:14.2427

## Work distribution:

Mihir Kubavat: Implemented the baseline and tested result using default.ipynb
Yash Serai: Made the report based on the default.py and default.ipynb as well as helped to prepare default.ipynb